In [1]:
#import modules and functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [1]:
import torch

CATH system nomenclature 
Column 1:  CATH domain name (seven characters)\
Column 2:  Class number\
Column 3:  Architecture number\
Column 4:  Topology number\
Column 5:  Homologous superfamily number

The domain names have seven characters (e.g. 1oaiA00)\
CHARACTERS 1-4: PDB Code\
The first 4 characters determine the PDB code e.g. 1oai

CHARACTER 5: Chain Character\
This determines which PDB chain is represented.\
Chain characters of zero ('0') indicate that the PDB file has no chain field.

CHARACTER 6-7: Domain Number\
The domain number is a 2-figure, zero-padded number (e.g. '01', '02' ... '10', '11', '12')\Where the domain number is a double ZERO ('00') this indicates that the domain is a whole PDB chain with no domain chopping. 

In [14]:
# create dataframe using data in the file
pdb =  pd.read_csv('core_database/destress_data_culled_pdb.csv',index_col=0)
pdb.columns = ['design_name','file_name','full_sequence','dssp_assignment',
               'composition_ALA','composition_CYS','composition_ASP','composition_GLU',
               'composition_PHE','composition_GLY','composition_HIS','composition_ILE','composition_LYS',
               'composition_LEU','composition_MET','composition_ASN','composition_PRO','composition_GLN',
               'composition_ARG','composition_SER','composition_THR','composition_VAL','composition_TRP',
               'composition_UNK','composition_TYR','ss_prop_alpha_helix','ss_prop_beta_bridge','ss_prop_beta_strand',
               'ss_prop_3_10_helix','ss_prop_pi_helix','ss_prop_hbonded_turn','ss_prop_bend','ss_prop_loop',
               'hydrophobic_fitness','isoelectric_point','charge','mass','num_residues','packing_density',
               'budeff_total','budeff_steric','budeff_desolvation','budeff_charge','evoef2_total','evoef2_ref_total',
               'evoef2_intraR_total','evoef2_interS_total','evoef2_interD_total','dfire2_total','rosetta_total',
               'rosetta_fa_atr','rosetta_fa_rep','rosetta_fa_intra_rep','rosetta_fa_elec','rosetta_fa_sol',
               'rosetta_lk_ball_wtd','rosetta_fa_intra_sol_xover4','rosetta_hbond_lr_bb','rosetta_hbond_sr_bb',
               'rosetta_hbond_bb_sc','rosetta_hbond_sc','rosetta_dslf_fa13','rosetta_rama_prepro','rosetta_p_aa_pp',
               'rosetta_fa_dun','rosetta_omega','rosetta_pro_close','rosetta_yhh_planarity',
               'aggrescan3d_total_value','aggrescan3d_avg_value','aggrescan3d_min_value','aggrescan3d_max_value']

#convert PDB CATH data from txt to csv
cath = pd.read_csv('/Users/Zachary/Desktop/Dissertation/CATH/CATH_daily_snapshot/cath-b-newest-all.txt', sep=' ',header = 0)
# Assign column names to the DataFrame
cath.columns = ["CATH domain name", 'version',"CATH num",'frag:chain']
cath.drop(columns= ['version','frag:chain'], inplace =True)
# Save the DataFrame as a CSV file
#cath.to_csv("cullpdb_pc95.0_res0.0-3.0_len40-10000_R0.3_Xray_d2023_02_13_chains47410.csv", index=False)

processing of PDB cath data

In [15]:
cath['PDB ID'] = cath['CATH domain name'].str[:-3]
cath['Chain'] = cath ['CATH domain name'].str[4:5]
#cath.drop(columns= 'verison',axis =0)

#find the rows with the same PDB ID but containing different CATH number: 930
cathnum_unique = ~cath.copy().duplicated(subset = 'CATH num',keep = False)
unique_cath = cath[cathnum_unique]
filtered_cath = unique_cath.drop_duplicates(subset = 'PDB ID',keep = 'first')

#after deleting the rows with unique CATH number, the remaining rows will contain one kind of CATH for each PDB ID
#2462 deleted

filtered_pdbid = filtered_cath['PDB ID'].unique()
rows_to_keep = ~cath['PDB ID'].isin(filtered_pdbid)
cath = cath[rows_to_keep]

#在此之后，我们就可以去掉重复的PDB ID，将两个dataframe合并 （现在有139436行）
cath.drop_duplicates(subset = 'PDB ID',keep = 'first',inplace = True)
cath.reset_index(drop = True, inplace = True)
cath.drop(columns= ['CATH domain name','Chain'], inplace =True)

merge cath and pdb according to the PDB ID

In [16]:
pdb['PDB ID'] = pdb ['design_name'].str[3:]
pdb.drop(columns= ['design_name', 'file_name',  'dssp_assignment', 
        'composition_ALA', 'composition_CYS', 'composition_ASP', 'composition_GLU', 'composition_PHE', 
        'composition_GLY', 'composition_HIS', 'composition_ILE', 'composition_LYS', 'composition_LEU', 
        'composition_MET', 'composition_ASN', 'composition_PRO', 'composition_GLN', 'composition_ARG', 
        'composition_SER', 'composition_THR', 'composition_VAL', 'composition_TRP', 'composition_UNK', 
        'composition_TYR', 'ss_prop_alpha_helix', 'ss_prop_beta_bridge', 'ss_prop_beta_strand', 
        'ss_prop_3_10_helix', 'ss_prop_pi_helix', 'ss_prop_hbonded_turn', 'ss_prop_bend', 'ss_prop_loop', 
        'hydrophobic_fitness', 'isoelectric_point', 'charge', 'mass', 'num_residues', 'packing_density', 
        'budeff_total', 'budeff_steric', 'budeff_desolvation', 'budeff_charge', 'evoef2_total', 
        'evoef2_ref_total', 'evoef2_intraR_total', 'evoef2_interS_total', 'evoef2_interD_total', 
        'dfire2_total', 'rosetta_total', 'rosetta_fa_atr', 'rosetta_fa_rep', 'rosetta_fa_intra_rep',
        'rosetta_fa_elec', 'rosetta_fa_sol', 'rosetta_lk_ball_wtd', 'rosetta_fa_intra_sol_xover4', 
        'rosetta_hbond_lr_bb', 'rosetta_hbond_sr_bb', 'rosetta_hbond_bb_sc', 'rosetta_hbond_sc',
        'rosetta_dslf_fa13', 'rosetta_rama_prepro', 'rosetta_p_aa_pp', 'rosetta_fa_dun', 'rosetta_omega',
        'rosetta_pro_close', 'rosetta_yhh_planarity', 'aggrescan3d_total_value', 'aggrescan3d_avg_value', 
        'aggrescan3d_min_value', 'aggrescan3d_max_value'],
         inplace =True)

In [17]:
finaldf = cath.merge(pdb, on = 'PDB ID', how = 'left')
#after checking the sequence by RCSBPDB-UNIPROT, I found that the sequence is correct for each PDB ID

In [18]:
#有10678行没有对应的fullsequence这里我们将他们去掉
nan_full = finaldf['full_sequence'].isnull()
finaldf_nan = finaldf[nan_full]
#finaldf_nan.head()
finaldf_nan
finaldf.dropna(subset = ['full_sequence'],inplace = True)

In [19]:
columns = finaldf.columns.tolist()

cathindex= columns.index('CATH num')
pdbindex = columns.index('PDB ID')
columns[pdbindex], columns[cathindex] = columns[cathindex], columns[pdbindex]
finaldf= finaldf[columns]
finaldf = finaldf.rename(columns = {
            'PDB ID':'Entry',
            'CATH num':'EC number',
            'full_sequence':'Sequence'
            })
finaldf.reset_index(drop = True, inplace = True)
finaldf.to_csv('cath_training_data_pisces.csv',sep = '\t',index = False)

In [ ]:
separation_index = len(finaldf)//2
df1 = finaldf.iloc[:separation_index]
df2 = finaldf.iloc[separation_index:]
#df1.to_csv('cath_training_data_pisces_1.csv',sep = '\t',index = False)
#df2.to_csv('cath_training_data_pisces_2.csv',sep = '\t',index = False)

compare the completeness of 3 different dataframe

In [2]:
finaldf = pd.read_csv('CLEAN/app/data/cath_training_data_pisces.csv', sep = '\t', index_col = False)

In [3]:
finaldf

,Entry,EC number,Sequence
0,12as,3.30.930.10,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...
1,12e8,2.60.40.10,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...
2,146l,1.10.530.40,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...
3,157l,1.10.530.40,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...
4,16pk,3.40.50.1260,EKKSINECDLKGKKVLIRVDFNVPVKNGKITNDYRIRSALPTLKKV...
...,...,...,...
32697,6vdd_b,1.20.1060.10,RGRALQPGELAAWLSEHSLGSRFGVAVVGTHKAYNADATALALVAA...
32698,6vdd_c,1.20.1060.10,RARALEPGELAAWLSEHSLGSRYGVAVVGTHKAYDADATALALVAA...
32699,6wgy_a,1.10.230.10,MTVVPENFVPGLDGVVAFTTEIAEPDKDGGALRYRGVDIEDLVSQR...
32700,6wgy_b,1.10.230.10,MTVVPQNFVPGLNGVVAFTTELAEPDKDGGALRYRGVDIEDLVSQR...


In [6]:
merged_first_two_digits_30000 = finaldf['EC number'].str.extract(r'(\d\.\d+\.\d+.\d+)')[0].sort_values().astype('category')
merged_first_two_digits_30000   

10321    1.10.10.10
26959    1.10.10.10
10973    1.10.10.10
13013    1.10.10.10
10975    1.10.10.10
            ...    
31550    6.20.80.10
31551    6.20.80.10
29239    6.20.90.10
29237    6.20.90.10
29238    6.20.90.10
Name: 0, Length: 32702, dtype: category
Categories (3008, object): ['1.10.10.10', '1.10.10.1070', '1.10.10.1080', '1.10.10.1100', ..., '6.20.50.110', '6.20.70.10', '6.20.80.10', '6.20.90.10']

In [ ]:

unique_categories_30000_list = list(merged_first_two_digits_30000.unique())
for index, category in enumerate(unique_categories_30000_list, start=1):
    print(f"{index}. {category}")


In [ ]:
merged_first_two_digits_30000 = finaldf['EC number'].str.extract(r'(\d\.\d+)')[0].sort_values().astype('category')

plt.figure(figsize=(12,6))
plt.hist(merged_first_two_digits_30000.values, bins='auto', edgecolor='black', rwidth=0.8)
plt.xticks(rotation=90) 
plt.xlabel('First two digits of CATH number')
plt.ylabel('Frequency')
plt.title('Histogram of CA in culled dataframe with 30000 data: STAGE3')
plt.show()

In [ ]:
fulllistdf = pd.read_csv('cath_training_data.csv',sep = '\t', index_col = False)

In [ ]:
fulllistdf

In [ ]:
unique_categories_128758_list = list(merge_first_two_digits_128758.unique())
for index, category in enumerate(unique_categories_128758_list, start=1):
    print(f"{index}. {category}")

In [ ]:
#to plot the CA histogram 
merge_first_two_digits_128758 = fulllistdf['EC number'].str.extract(r'(\d\.\d+)')[0].sort_values().astype('category')

plt.figure(figsize=(12,6))
plt.hist(merge_first_two_digits_128758.values, bins ='auto',edgecolor = 'black',rwidth = 0.8)
plt.xlabel('First two digits of CATH number')
plt.ylabel('Frequency')
plt.title ('Histogram of CA in merged dataframe with 128758 dots: STAGE2')
plt.show()

In [ ]:
merge_first_two_digits_128758

In [ ]:
#to plot the CA histogram 
cath_first_two_digits = cath['CATH num'].str.extract(r'(\d\.\d+\.\d+)')[0].sort_values().astype('category')


plt.figure(figsize=(12,6))
plt.hist(cath_first_two_digits.values, bins ='auto',edgecolor = 'black',rwidth = 0.8)
plt.xlabel('First two digits of CATH number')
plt.ylabel('Frequency')
plt.title ('Histogram of CA in full dataframe: STAGE1')
plt.show()

In [ ]:
cath_first_two_digits

trim the csv by filtering through the existing pth files

In [34]:
import os

# Set the directory path
dir_path = "/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/esm_data"

# List all files in the directory
all_files = os.listdir(dir_path)

# Filter the files with the .pt extension
pt_files = [f for f in all_files if f.endswith(".pt")]

# Remove the .pt extension from each file name and create a list
file_names_without_ext = [f[:4] for f in pt_files]

# Print the list length and the first few elements
print(f"Number of files: {len(file_names_without_ext)}")
print("First few elements:", file_names_without_ext[:10])


Number of files: 35767
First few elements: ['5xgt', '2q2v', '2af4', '5mbm', '5t5i', '2qsv', '1ykh', '3k8a', '4adb', '3hba']


In [36]:
#find duplicates in the esm_data folder
def find_duplicates(file_names_without_ext):
    duplicates = []
    seen = set()

    for file_name in file_names_without_ext:
        if file_name in seen:
            duplicates.append(file_name)
        else:
            seen.add(file_name)

    return duplicates
duplicates = find_duplicates(file_names_without_ext)

#print("Duplicates found:", duplicates)
len(duplicates)


4075

Mutate the sequences with orphan CATH number, so that each CATH number contains at least more than one sequence

In [39]:
finaldf

,Entry,EC number,Sequence
0,12as,3.30.930.10,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...
1,12e8,2.60.40.10,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...
2,146l,1.10.530.40,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...
3,157l,1.10.530.40,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...
4,16pk,3.40.50.1260,EKKSINECDLKGKKVLIRVDFNVPVKNGKITNDYRIRSALPTLKKV...
...,...,...,...
29982,7bwf,3.30.2310.20,ARLNITFSPQAFEDYKYFQQNNKKMVKKINELLKSIDRNGALEGIG...
29983,7bwg,3.20.20.80,LPAVVPAPAAIEQATGAPFRLDASTRIEGEADAASALSALLEARTG...
29984,7bwh,3.10.120.10,EEQTFSWSEISQHTSANSLWVVVRDKTSPGSPLRVYDVTNFQKTHP...
29985,7c4a,3.50.50.60,GFDYDVVVVGGGFAGATAARECGLQGYRTLLLEARSRLGGRTFTSR...


In [9]:
unique_rows = finaldf[~finaldf['EC number'].duplicated(keep=False)]
print(unique_rows)


# First, get the count of each EC number
ec_counts = finaldf['EC number'].value_counts()

# Get the list of non-unique EC numbers
non_unique_ec_numbers = ec_counts[ec_counts > 1].index

# Filter the finaldf DataFrame to include only rows with non-unique EC numbers
finaldf = finaldf[finaldf['EC number'].isin(non_unique_ec_numbers)]
finaldf


      Entry      EC number                                           Sequence
52     1a73     3.90.75.10  ALTNAQILAVIDSWEETVGQFPVITHHVPLGGGLQGTLHCYEIPLA...
70     1a92    4.10.220.40  GREDILEQWVSGRKKLEELERDLRKLKKKIKKLEEDNPWLGNIKGI...
90     1af7    1.10.155.10  SVLLQMTQRLALSDAHFRRICQLIYQRAGIVLADHKRDMVYNRLVR...
158    1avo     1.20.5.120  LRVQPEAQAKVDVFREDLCTKTENLLGSYFPKKISELDAFLKEPAL...
257    1bg1   1.20.1050.20  VVTEKQQMLEQHLQDVRKRVQDLEQKMKVVENLQDDFDFNYKTLKS...
...     ...            ...                                                ...
29752  6te6    1.10.260.60  LELRLKSPVGAEPAVYPWPLPVYDKHHDAAHEIIETIRWVCEEIPD...
29757  6tm3  3.40.50.10280  SKKLLFQFDTDATPSVFDVVVGYDGGADHITGYGNVTPDNVGAYVD...
29836  6uw5     1.10.8.240  MKITVLVGGVGGARFLLGVQNLLGLGSFADGPSKHELTAVVNIGDD...
29870  6vdd   1.20.1060.10  RGRALEPGELAAWLSEHSLGSRFGVAVVGTHKAYDADATALAIVAA...
29920  6wgy    1.10.230.10  MTVVPENFVPGLDGVVAFTTEIAEPDKDGGALRYRGVDIEDLVSQR...

[1359 rows x 3 columns]


,Entry,EC number,Sequence
0,12as,3.30.930.10,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...
1,12e8,2.60.40.10,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...
2,146l,1.10.530.40,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...
3,157l,1.10.530.40,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...
4,16pk,3.40.50.1260,EKKSINECDLKGKKVLIRVDFNVPVKNGKITNDYRIRSALPTLKKV...
...,...,...,...
29982,7bwf,3.30.2310.20,ARLNITFSPQAFEDYKYFQQNNKKMVKKINELLKSIDRNGALEGIG...
29983,7bwg,3.20.20.80,LPAVVPAPAAIEQATGAPFRLDASTRIEGEADAASALSALLEARTG...
29984,7bwh,3.10.120.10,EEQTFSWSEISQHTSANSLWVVVRDKTSPGSPLRVYDVTNFQKTHP...
29985,7c4a,3.50.50.60,GFDYDVVVVGGGFAGATAARECGLQGYRTLLLEARSRLGGRTFTSR...


In [ ]:
columns = unique_rows.columns
triplicated_rows = pd.DataFrame(columns=columns)

def change_first_occurrences(sequence, changes):
    new_sequence = list(sequence)
    for old_aa, new_aa in changes:
        try:
            position = new_sequence.index(old_aa)
            new_sequence[position] = new_aa
        except ValueError:
            pass
    return ''.join(new_sequence)

for index, row in unique_rows.iterrows():
    entry = row['Entry']
    ec_number = row['EC number']
    sequence = row['Sequence']

    new_row_a = pd.Series({'Entry': f"{entry}_a", 'EC number': ec_number, 'Sequence': sequence}, name=index)
    new_row_b = pd.Series({'Entry': f"{entry}_b", 'EC number': ec_number, 'Sequence': change_first_occurrences(sequence, [('E', 'Q'), ('D', 'N'), ('I', 'L')])}, name=index)
    new_row_c = pd.Series({'Entry': f"{entry}_c", 'EC number': ec_number, 'Sequence': change_first_occurrences(sequence, [('F', 'Y'), ('G', 'A'), ('I', 'L')])}, name=index)

    triplicated_rows = triplicated_rows.append([new_row_a, new_row_b, new_row_c])

print(triplicated_rows)
triplicated_rows.to_csv('CLEAN/app/data/cath_training_data_pisces_triplicated.csv',sep = '\t',index = False)


In [ ]:
#modify the finaldf after generating mutated sequences: remove unique rows and add the triplicated rows 
#finaldf = pd.read_csv('CLEAN/app/data/cath_training_data_pisces.csv', sep = '\t', index_col = False)
unique_entries = unique_rows['Entry']
finaldf = finaldf[~finaldf['Entry'].isin(unique_entries)]

# Concatenate triplicated_rows to the bottom of finaldf
finaldf = pd.concat([finaldf, triplicated_rows], ignore_index=True)
finaldf.to_csv('CLEAN/app/data/cath_training_data_pisces.csv',sep = '\t',index = False)

In the triplicated rows, there are sequences longer than 1024, we need to trim them for retrieving embeddings

In [ ]:
#find the long sequence in the triplicated rows
triplicated_long_mask = triplicated_rows['Sequence'].str.len()>1024
triplicated_long_sequence = triplicated_rows[triplicated_long_mask]
triplicated_long_sequence.reset_index(drop = True, inplace = True)
triplicated_long_sequence

# Initialize an empty DataFrame to store the new chunked rows
chunked_triplicated_long_sequence = pd.DataFrame(columns=['Entry', 'EC number', 'Sequence'])

# Set the chunk size and overlap
chunk_size = 800

# Iterate through the rows in long_sequences
for index, row in triplicated_long_sequence.iterrows():
    entry = row['Entry']
    ec_number = row['EC number']
    sequence = row['Sequence']
    
    # Calculate the number of chunks
    num_chunks = (len(sequence)) // chunk_size + 1
    
    # Create chunks and add them to chunked_sequences_df
    for i in range(num_chunks):
        start = i * chunk_size
        end = start + chunk_size
        
        chunked_entry = f"{entry}_{i+1}"
        chunked_sequence = sequence[start:end]
        
        chunked_triplicated_long_sequence = chunked_triplicated_long_sequence.append({'Entry': chunked_entry, 
                                                            'EC number': ec_number, 
                                                            'Sequence': chunked_sequence}, 
                                                           ignore_index=True)


In [ ]:
chunked_triplicated_long_sequence.to_csv('CLEAN/app/data/cath_training_data_pisces_triplicated_long.csv',sep = '\t',index = False)

i have a dataframe called finaldf, with three columns Entry,	EC number,	Sequence. I want to delete the rows in this dataframe depends on the data under Entry. If the data is present in the list 'file_names_without_ext', the corresponding row should be removed from the finaldf

In [ ]:
# Create a boolean mask for the rows to keep
mask = ~finaldf['Entry'].isin(file_names_without_ext)
filtered_finaldf = finaldf[mask]
filtered_finaldf.reset_index(drop = True, inplace = True)
filtered_finaldf.shape

In [ ]:
short_mask = filtered_finaldf['Sequence'].str.len()<=1024
long_mask = filtered_finaldf['Sequence'].str.len()>1024

long_sequence = filtered_finaldf[long_mask]
short_sequence = filtered_finaldf[short_mask]

long_sequence.reset_index(drop = True, inplace = True)
short_sequence.reset_index(drop = True, inplace = True)

In [ ]:
category_of_long = long_sequence['EC number'].str.extract (r'(\d\.\d+)')[0].sort_values().astype('category')
#category_of_long = list(category_of_long.unique())
#for index, category in enumerate(categoryssssss, start=1):
    #print(f"{index}. {category}")

In [ ]:
#to plot the CA histogram

plt.figure(figsize=(24,6))
plt.hist(category_of_long, bins ='auto',edgecolor = 'black',rwidth = 0.8)
plt.xlabel('First two digits of CATH number')
plt.ylabel('Frequency')
plt.title ("Histogram of long sequences'category")
plt.show()

In [ ]:
long_sequence

In [ ]:
plt.figure(figsize=(24,6))
plt.hist(long_sequence['Sequence'].str.len(), bins ='auto',edgecolor = 'black',rwidth = 0.8)
plt.xlabel('First two digits of CATH number')
plt.ylabel('Frequency')
plt.title ("Histogram of long sequences'category")
plt.show()

so I want to chunk the Sequence under long_sequence.
so take row 1 as an example. the original entry name is 1a0c. the row1['Sequence'].str.len()= 1025
after chopping, two new rows were created and add to a new dataframe.
the first row has Entry as 1a0c_1, Sequence as the first 800 characters of the original Sequence.
the second row has Entry as 1a0c_2, Sequence as the remaining characters of the original Sequence.

In [ ]:
# Initialize an empty DataFrame to store the new chunked rows
chunked_long_sequences = pd.DataFrame(columns=['Entry', 'EC number', 'Sequence'])

# Set the chunk size and overlap
chunk_size = 800

# Iterate through the rows in long_sequences
for index, row in long_sequence.iterrows():
    entry = row['Entry']
    ec_number = row['EC number']
    sequence = row['Sequence']
    
    # Calculate the number of chunks
    num_chunks = (len(sequence)) // chunk_size + 1
    
    # Create chunks and add them to chunked_sequences_df
    for i in range(num_chunks):
        start = i * chunk_size
        end = start + chunk_size
        
        chunked_entry = f"{entry}_{i+1}"
        chunked_sequence = sequence[start:end]
        
        chunked_long_sequences = chunked_long_sequences.append({'Entry': chunked_entry, 
                                                            'EC number': ec_number, 
                                                            'Sequence': chunked_sequence}, 
                                                           ignore_index=True)


In [ ]:
plt.figure(figsize=(24,6))
plt.hist(chunked_long_sequences['Sequence'].str.len(), bins ='auto',edgecolor = 'black',rwidth = 0.8)
plt.xlabel('First two digits of CATH number')
plt.ylabel('Frequency')
plt.title ("Histogram of long sequences'category")
plt.show()

#合并chunked_long_sequences 和 short_sequences

In [ ]:
cs_merged = pd.concat([short_sequence, chunked_long_sequences], ignore_index=True)
cs_merged.to_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/cath_training_data_chunked.csv', sep = '\t', index = False)

跑embeddings遇到问题 有16个sequence长度在1023-1024左右但跑不出来

In [ ]:
# Initialize an empty DataFrame to store the new chunked rows
chunked_16_sequences = pd.DataFrame(columns=['Entry', 'EC number', 'Sequence'])

# Set the chunk size and overlap
chunk_size = 800

# Iterate through the rows in long_sequences
for index, row in filtered_finaldf.iterrows():
    entry = row['Entry']
    ec_number = row['EC number']
    sequence = row['Sequence']
    
    # Calculate the number of chunks
    num_chunks = (len(sequence)) // chunk_size + 1
    
    # Create chunks and add them to chunked_sequences_df
    for i in range(num_chunks):
        start = i * chunk_size
        end = start + chunk_size
        
        chunked_entry = f"{entry}_{i+1}"
        chunked_sequence = sequence[start:end]
        
        chunked_16_sequences = chunked_16_sequences.append({'Entry': chunked_entry, 
                                                            'EC number': ec_number, 
                                                            'Sequence': chunked_sequence}, 
                                                           ignore_index=True)


In [ ]:
chunked_16_sequences.to_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/cath_training_data_chunked_last16.csv', sep = '\t', index = False)

In [ ]:
all_files

In [34]:
content = torch. load('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/model/short_cath_training_data_pisces_4500.pth')
content#['mean_representations'][33]

OrderedDict([('fc1.weight',
              tensor([[-0.0405,  0.0150, -0.0203,  ..., -0.0193, -0.0486,  0.0687],
                      [ 0.0407,  0.0625,  0.0365,  ..., -0.0331, -0.0281,  0.0447],
                      [ 0.0593, -0.0682,  0.0888,  ..., -0.0439,  0.0716,  0.0633],
                      ...,
                      [ 0.0834,  0.0174, -0.0658,  ..., -0.0715, -0.0565,  0.0564],
                      [ 0.0215, -0.0199, -0.0058,  ..., -0.0181, -0.0052,  0.0607],
                      [ 0.0106,  0.0650,  0.0281,  ..., -0.0337, -0.0040,  0.0074]])),
             ('fc1.bias',
              tensor([ 3.7993e-02,  4.5841e-02,  1.9512e-02,  9.9381e-03,  9.6407e-03,
                       2.8563e-02,  1.4436e-03,  2.1185e-02,  3.2070e-02,  2.9989e-02,
                       3.4130e-02,  3.9551e-02,  3.4568e-02,  1.0014e-02,  2.6828e-02,
                       4.0956e-02,  1.1983e-02,  4.2441e-02,  4.1630e-02, -8.5051e-03,
                       1.3203e-03, -2.6323e-03, -1.2790e-02,  9.

In [ ]:
content_1 = torch. load('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/esm_data/1a0c_3.pt')
content_1

In [18]:
cath_training_data_pisces= pd.read_csv('CLEAN/app/data/cath_training_data_pisces.csv', sep = '\t')

In [19]:

# Drop the 'Unnamed: 0' column
#cath_training_data_pisces.drop(columns=['Unnamed: 0'], inplace=True)
cath_training_data_pisces


,Entry,EC number,Sequence
0,12as,3.30.930.10,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...
1,12e8,2.60.40.10,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...
2,146l,1.10.530.40,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...
3,157l,1.10.530.40,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...
4,16pk,3.40.50.1260,EKKSINECDLKGKKVLIRVDFNVPVKNGKITNDYRIRSALPTLKKV...
...,...,...,...
32698,6vdd_b,1.20.1060.10,RGRALQPGELAAWLSEHSLGSRFGVAVVGTHKAYNADATALALVAA...
32699,6vdd_c,1.20.1060.10,RARALEPGELAAWLSEHSLGSRYGVAVVGTHKAYDADATALALVAA...
32700,6wgy_a,1.10.230.10,MTVVPENFVPGLDGVVAFTTEIAEPDKDGGALRYRGVDIEDLVSQR...
32701,6wgy_b,1.10.230.10,MTVVPQNFVPGLNGVVAFTTELAEPDKDGGALRYRGVDIEDLVSQR...


In [10]:
import os

# Set the path to the esm_data folder
esm_data_path = '/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/esm_data'

# Get the list of files in the esm_data folder
esm_data_files = os.listdir(esm_data_path)

# Check if each Entry exists in the esm_data folder
entries_not_found = []
for entry in cath_training_data_pisces['Entry']:
    # Create the file name with '.pt' extension
    file_name = f'{entry}.pt'
    
    if file_name not in esm_data_files:
        entries_not_found.append(entry)

if not entries_not_found:
    print("All entries found in the esm_data folder.")
else:
    print(f"The following entries were not found in the esm_data folder: {entries_not_found}")


The following entries were not found in the esm_data folder: ['6fkm_b', '6fkm_c']


In [11]:
# List of entries to remove
entries_to_remove = ['6fkm_b', '6fkm_c']

# Keep only rows that have Entry values not in the entries_to_remove list
cath_training_data_pisces = cath_training_data_pisces[~cath_training_data_pisces['Entry'].isin(entries_to_remove)]

# Reset the index after removing rows (optional)
cath_training_data_pisces.reset_index(drop=True, inplace=True)


In [12]:
# Check if each Entry exists in the esm_data folder
entries_not_found = []
for entry in cath_training_data_pisces['Entry']:
    # Create the file name with '.pt' extension
    file_name = f'{entry}.pt'
    
    if file_name not in esm_data_files:
        entries_not_found.append(entry)

if not entries_not_found:
    print("All entries found in the esm_data folder.")
else:
    print(f"The following entries were not found in the esm_data folder: {entries_not_found}")


All entries found in the esm_data folder.


In [17]:
cath_training_data_pisces.to_csv('CLEAN/app/data/cath_training_data_pisces.csv', sep = '\t',index=False)

In [22]:
cath_training_data_pisces_without_unique = cath_training_data_pisces.iloc[:28628]
cath_training_data_pisces_without_unique.to_csv('CLEAN/app/data/cath_training_data_pisces_without_unique.csv', sep = '\t',index=False)

In [29]:
import os

directory = '/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/esm_data'
length=[]
for filename in os.listdir(directory):
    if filename.endswith('.pt'):
        file_path = os.path.join(directory, filename)
        data = torch.load(file_path)
        label = data['label']
        label_length = len(label)
        length.append(label_length)
        #print(f"Label: {label}, Length: {label_length}")



NameError: name 'lengths' is not defined

In [35]:
cath_training_data_pisces = pd.read_csv('CLEAN/app/data/cath_training_data_pisces.csv', sep = '\t')

# List of entries to remove
entries_to_remove = ['6fkm_a']

# Keep only rows that have Entry values not in the entries_to_remove list
cath_training_data_pisces = cath_training_data_pisces[~cath_training_data_pisces['Entry'].isin(entries_to_remove)]

# Reset the index after removing rows (optional)
cath_training_data_pisces.reset_index(drop=True, inplace=True)
cath_training_data_pisces.to_csv('CLEAN/app/data/cath_training_data_pisces.csv', sep = '\t',index=False)



In [34]:
a.nunique()

2

Prepare the data for inferring from the trained model

In [2]:
import pandas as pd

processing of finaldf

In [3]:
finaldf = pd.read_csv('cath_training_data_pisces.csv', sep = '\t')
mask = finaldf['Sequence'].str.len() < 1023
short_cath_training_data_pisces = finaldf[mask]
# Assuming finaldf is already defined and has a column 'EC_number'
# First, get the count of each EC number
ec_counts = short_cath_training_data_pisces['EC number'].value_counts()

# Get the list of unique EC numbers (i.e., EC numbers that occur only once)
unique_ec_numbers = ec_counts[ec_counts == 1].index

# Filter the finaldf DataFrame to exclude rows with unique EC numbers
short_cath_training_data_pisces = short_cath_training_data_pisces[~short_cath_training_data_pisces['EC number'].isin(unique_ec_numbers)]
short_cath_training_data_pisces.to_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/short_cath_training_data_pisces.csv', sep = '\t',index=False)


In [36]:
short_cath_training_data_pisces = pd.read_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/short_cath_training_data_pisces.csv', sep = '\t')

In [7]:
# Perform an outer join with indicator=True
combined_df = finaldf.merge(short_cath_training_data_pisces, how='outer', indicator=True)

# Filter the combined DataFrame to only keep rows from 'finaldf' that are not in 'short_cath_training_data_pisces'
extra_cath_training_data_pisces = combined_df[combined_df['_merge'] == 'left_only'].drop(columns=['_merge'])

# First, get the count of each EC number
ec_counts = extra_cath_training_data_pisces['EC number'].value_counts()

# Get the list of unique EC numbers (i.e., EC numbers that occur only once)
unique_ec_numbers = ec_counts[ec_counts == 1].index

# Filter the finaldf DataFrame to exclude rows with unique EC numbers
extra_cath_training_data_pisces = extra_cath_training_data_pisces[~extra_cath_training_data_pisces['EC number'].isin(unique_ec_numbers)]
extra_cath_training_data_pisces.reset_index(drop=True, inplace=True)

extra_cath_training_data_pisces.to_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/extra_cath_training_data_pisces.csv', sep = '\t',index=False)

选出不在training set里的PDBstructures

In [21]:
cath_training_data = pd.read_csv('cath_training_data.csv', sep = '\t')
short_cath_training_data_pisces = pd.read_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/short_cath_training_data_pisces.csv', sep = '\t')
#filter the rows in cath_training_data_pisces that have a sequence length less than 1023
mask_1 = cath_training_data['Sequence'].str.len() < 1023
short_cath_training_data = cath_training_data[mask_1]

# Trim the 'Entry' column in cath_training_data_pisces to 4 characters
#cath_training_data_pisces['Entry'] = cath_training_data_pisces['Entry'].apply(lambda x: x[:4])

# Merge the two dataframes on the common columns, with indicator=True
merged_data = short_cath_training_data.merge(short_cath_training_data_pisces, 
                                        on=['Entry', 'EC number', 'Sequence'],
                                        how='outer', indicator=True)

# Filter the rows that are present only in the 'cath_training_data' dataframe
unique_short_cath_training_data = merged_data[merged_data['_merge'] == 'left_only']

# Drop the '_merge' column as it's no longer needed
unique_short_cath_training_data = unique_short_cath_training_data.drop(columns=['_merge'])

# Reset index of the resulting dataframe
unique_short_PDB_cath_training_data = unique_short_cath_training_data.reset_index(drop=True)

# Now, unique_cath_training_data contains the rows present in 'cath_training_data' but not in 'cath_training_data_pisces' after trimming the 'Entry' column
#the next step is to create a new dataframe with the rows whose seuqnce is shorter than 1023 and 

#then we need to separate the rows in the short_unique_cath_training_data dataframe into two dataframes,
#one with the rows that have the existing EC number as the cath_training_data_pisces, one does not. 

# Get the unique EC numbers from the cath_training_data_pisces dataframe
pisces_ec_numbers = short_cath_training_data_pisces['EC number'].unique()

# Create a mask with True values for rows with EC numbers existing in cath_training_data_pisces
ec_number_mask = unique_short_PDB_cath_training_data['EC number'].isin(pisces_ec_numbers)

# Apply the mask to the filtered_unique_cath_training_data dataframe to get the rows with existing EC numbers
existing_ec_numbers_df = unique_short_PDB_cath_training_data[ec_number_mask] #79015
existing_ec_numbers_df.reset_index(drop=True, inplace=True)

# Invert the mask to get the rows with non-existing EC numbers
non_existing_ec_numbers_df = unique_short_PDB_cath_training_data[~ec_number_mask] #5473
non_existing_ec_numbers_df.reset_index(drop=True, inplace=True)

选出合适的AF2structure

In [33]:
cath_training_data_af2 = pd.read_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/cath_training_data_af2.csv', sep = '\t')#raw data
cath_assign_data_af2 = pd.read_csv('/Users/Zachary/Desktop/Dissertation/CATH/cath-v4_3_0.alphafold-v2.2022-11-22.tsv', sep = '\t')#labeled data

In [35]:
#filter the rows in cath_assign_data_af2 that have a sequence length less than 1023
mask_2 = cath_training_data_af2['Sequence'].str.len() < 1023
short_cath_training_data_af2= cath_training_data_af2[mask_2]

In [36]:
# Keep only the 'domain_ID' and 'sfam_id' columns
cath_assign_data_af2 = cath_assign_data_af2[['domain_ID', 'sfam_id']]

# Create a new column 'Entry' by extracting the value between the first two underscores in 'domain_ID'
cath_assign_data_af2['Entry'] = cath_assign_data_af2['domain_ID'].str.split('_').str[1]

# Create a new column 'EC_number' with the same values as the 'sfam_id' column
cath_assign_data_af2['EC number'] = cath_assign_data_af2['sfam_id']
cath_assign_data_af2 = cath_assign_data_af2[['Entry', 'EC number']]

In [37]:
# Read the CSV file of the destress af2 data
destress_data_af2 = pd.read_csv('/Users/Zachary/Desktop/Dissertation/core_database/destress_data_af2.csv')

# Keep only the 'design_name' and 'full_sequence' columns
destress_data_af2 = destress_data_af2[['design_name', 'full_sequence']]

destress_data_af2['Entry'] = destress_data_af2['design_name'].str.split('-').str[1]
destress_data_af2['Sequence'] = destress_data_af2['full_sequence']
destress_data_af2 = destress_data_af2[['Entry', 'Sequence']]

cath_assign_data_af2 = pd.merge(cath_assign_data_af2, destress_data_af2, on='Entry')

#filter the rows in cath_assign_data_af2 that have a sequence length less than 1023
mask_3 = cath_assign_data_af2['Sequence'].str.len() < 1023
short_cath_assign_data_af2= cath_assign_data_af2[mask_3]

In [38]:
# Get the list of 'Entry' values from short_cath_assign_data_af2
entry_values = short_cath_assign_data_af2['Entry'].unique()

# Create a boolean mask indicating if 'Entry' values in short_cath_training_data_af2 are in the entry_values list
mask_4 = short_cath_training_data_af2['Entry'].isin(entry_values)

# Separate the cath_training_data_af2 DataFrame into two DataFrames based on the mask 525478 intotal
present_in_assign = short_cath_training_data_af2[mask_4]#193262
not_present_in_assign = short_cath_training_data_af2[~mask_4]#332216

Randomly sample some data to be the inferring file

In [11]:
# Sample 10 random rows from existing_ec_numbers_df
same_test_1 = existing_ec_numbers_df.sample(n=500, random_state=42)

# Reset index of the resulting dataframe
same_test_1 = same_test_1.reset_index(drop=True)

same_test_1.to_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/same_test_1.csv', sep = '\t',index=False)

In [12]:
# Sample 10 random rows from existing_ec_numbers_df
same_test_2 = non_existing_ec_numbers_df.sample(n=500, random_state=42)

# Reset index of the resulting dataframe
same_test_2 = same_test_2.reset_index(drop=True)

same_test_2.to_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/same_test_2.csv', sep = '\t',index=False)

In [43]:
# Sample 10 random rows from existing_ec_numbers_df
same_test_3 = present_in_assign.sample(n=500, random_state=42)

# Reset index of the resulting dataframe
same_test_3 = same_test_3.reset_index(drop=True)

same_test_3.to_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/same_test_3.csv', sep = '\t',index=False)

 

In [44]:
# Sample 10 random rows from existing_ec_numbers_df
same_test_4 = not_present_in_assign.sample(n=500, random_state=42)

# Reset index of the resulting dataframe
same_test_4 = same_test_4.reset_index(drop=True)

same_test_4.to_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/same_test_4.csv', sep = '\t',index=False)

 

create a new csv with long pisces data

In [51]:
finaldf = pd.read_csv('cath_training_data_pisces.csv', sep = '\t')
# First, get the count of each EC number
ec_counts = finaldf['EC number'].value_counts()

# Get the list of unique EC numbers (i.e., EC numbers that occur only once)
unique_ec_numbers = ec_counts[ec_counts == 1].index

# Filter the finaldf DataFrame to exclude rows with unique EC numbers
finaldf = finaldf[~finaldf['EC number'].isin(unique_ec_numbers)]

mask = finaldf['Sequence'].str.len() >= 1023
long_cath_training_data_pisces = finaldf[mask]
long_cath_training_data_pisces.to_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/long_cath_training_data_pisces.csv', sep = '\t',index=False)




In [50]:
long_cath_training_data_pisces

,Entry,EC number,Sequence
11,1a0c,3.20.20.150,NKYFENVSKIKYEGPKSNNPYSFKFYNPEEVIDGKTMEEHLRFSIA...
12,1a0d,3.20.20.150,PYFDNISTIAYEGPASKNPLAFKFYNPEEKVGDKTMEEHLRFSVAY...
18,1a0t,2.40.170.10,SGFEFHGYARSGVIMNDSGASTKSGAYITPAGETGGAIGRLGNQAD...
19,1a12,2.130.10.30,KKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENVMERKKPALVSIP...
34,1a4e,2.40.180.10,DVREDRVVTNSTGNPINEPFVTQRIGEHGPLLLQDYNLIDSLAHFN...
...,...,...,...
29967,6y64,2.60.175.10,ITEIEAYLNPRMGQPQNEDFYGFSDNVTVSDDFGSDAPPWKQFPCY...
29968,6y9t,3.20.20.80,MSHWYDHAIIYQIYPKSFQDSNDDGIGDLNGIRKRIPYLQNLGVNA...
29971,6yei,3.40.50.10860,NALAATNRNFKLAARLLGLDSKLEKSLLIPFREIKVECTIPKDDGT...
29973,6ypa,3.60.110.10,HMVKVGYIQMEPKILELDKNYSKAEKLIKEASKEGAKLVVLPELFD...


In [ ]:
#original function
def mine_hard_negative(dist_map, knn=10):
    #print("The number of unique EC numbers: ", len(dist_map.keys()))
    ecs = list(dist_map.keys())
    negative = {}
    for i, target in enumerate(ecs):
        sort_orders = sorted(
            dist_map[target].items(), key=lambda x: x[1], reverse=False)
        if sort_orders[1][1] != 0:
            neg_ecs = [i[0] for i in sort_orders[1:1 + knn]]
        elif sort_orders[2][1] != 0:
            freq = [1/i[1] for i in sort_orders[2:2+knn]]
            neg_ecs = [i[0] for i in sort_orders[2:2+knn]]
        elif sort_orders[3][1] != 0:
            freq = [1/i[1] for i in sort_orders[3:3+knn]]
            neg_ecs = [i[0] for i in sort_orders[3:3+knn]]
        else:
            freq = [1/i[1] for i in sort_orders[4:4+knn]]
            neg_ecs = [i[0] for i in sort_orders[4:4+knn]]

        normalized_freq = [i/sum(freq) for i in freq]
        
        negative[target] = {
            'weights': normalized_freq,
            'negative': neg_ecs
        }
    return negative

In [47]:
finaldf

NameError: name 'finaldf' is not defined

,Entry,EC number,Sequence
0,6tob,1.10.8.50,GSHMVALPQLTDEQRAAALEKAAAARRARAELKDRLKRGGTNLTQV...
1,5qzq,2.60.34.20,GAMNSSNYAELFNNDIKLFVDDTNVYRVTVHKTFEGNVATKAINGC...
2,1zcd,1.20.1530.10,SSDASGGIILIIAAILAMIMANSGATSGWYHDFLETPVQLRVGSLE...
3,1w4j,4.10.320.10,GSREVAAMPAARRLAKELGIDASKVKGTGPGGVITVEDVKRWAEET...
4,4ew5,3.10.450.160,HRKNGGKPDHVESDISYAVARQLAVNLGLTGYQSLPPGIAKNLARG...
...,...,...,...
495,4e1f,1.10.3680.20,ATPDFPTHFPKSSIGIQNELAGLVVAMPANSAQKFGYVKSAQGDAL...
496,1f35,2.60.120.390,AEDGPQKQQLEPLVLDQDLTQQRLRVESLKQRGEKKQDGEKLIRPA...
497,5qy1,2.60.34.20,GAMNSSNYAELFNNDIKLFVDDTNVYRVTVHKTFEGNVATKAINGC...
498,4dq7,1.10.530.50,LQALLPKNSRVRFTTAEVDSAVARISQKIGVPASYYQFLIPIENFV...


94     1.10.10.1080
320    1.10.10.1250
482    1.10.10.1400
18     1.10.10.1420
213    1.10.10.2150
           ...     
247    6.10.250.590
379     6.10.280.60
258     6.10.280.80
381     6.20.220.10
78      6.20.350.10
Name: 0, Length: 500, dtype: category
Categories (378, object): ['1.10.10.1080', '1.10.10.1250', '1.10.10.1400', '1.10.10.1420', ..., '6.10.280.60', '6.10.280.80', '6.20.220.10', '6.20.350.10']

In [56]:
sample_1 = pd.read_csv ('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/same_test_3.csv', sep = '\t')
finaldf = pd.read_csv('CLEAN/app/data/cath_training_data_pisces.csv', sep = '\t', index_col = False)

finaldf1 = finaldf['EC number'].str.extract(r'(\d)')[0].sort_values().astype('category')
sample1  = sample_1['EC number'].str.extract(r'(\d)')[0].sort_values().astype('category')

In [57]:

# Get unique EC numbers in finaldf and sample_1
finaldf_unique_ec_numbers = set(finaldf1)
sample_1_unique_ec_numbers = set(sample1)

# Find the intersection of the two sets
common_ec_numbers = finaldf_unique_ec_numbers.intersection(sample_1_unique_ec_numbers)

# Calculate the number of unique EC numbers in the intersection
number_of_common_ec_numbers = len(common_ec_numbers)

print(f'There are {number_of_common_ec_numbers} unique EC numbers in sample_1 that can be found in finaldf.')


There are 1 unique EC numbers in sample_1 that can be found in finaldf.


In [64]:
sample_1 = pd.read_csv ('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/same_test_2.csv', sep = '\t')
sample1 = sample_1['EC number'].str.extract(r'(\d\.\d+)')[0].sort_values().astype('category')
sample1

0      1.10
356    1.10
134    1.10
358    1.10
128    1.10
       ... 
359    6.10
377    6.10
37     6.10
78     6.20
381    6.20
Name: 0, Length: 500, dtype: category
Categories (21, object): ['1.10', '1.20', '1.25', '2.10', ..., '3.90', '4.10', '6.10', '6.20']